In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import re
from sklearn.metrics import confusion_matrix

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = list(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
manual_input=["say","worker","company","said"]
stop_words=stop_words+manual_input

In [ ]:
lemmatizer.lemmatize("abusing")

'abusing'

In [ ]:
porter.stem("abusing")

'abus'

In [ ]:
def preprocessing(document):
    # Remove punctuation and numbers
    try:
        
      document = re.sub(r"[^A-Za-z\u00c0-\u017e\s]+", " ", document)

      # Remove extra white spaces
      doc= re.sub(r"[\s]{2,}", " ", document)
      
      doc = doc.lower()
      doc = word_tokenize(doc)
      
      doc = [lemmatizer.lemmatize(word) for word in doc]
      doc = [word for word in doc if word not in stop_words]
      doc = [porter.stem(word) for word in doc]
    except:
      doc=[]
    return doc

In [ ]:
#load data file :File should contain column names title, id, content 
data= pd.read_csv("/content/drive/MyDrive/Chain_News_Monitoring_Project/Pujit/training_data.csv")
data.drop(columns=["Unnamed: 0"],inplace=True)
data

,id,ts,source,title,content,confirmedLabel
0,abc.es-1c3ef10282721375df4426f954bf89cd8d96f57f,1664446706000,abc.es,"""Mauricio Claver-Carone former IDB President:...","""Latin America's largest investment bank is in...",1
1,abc.es-9067af483ba8f35028c6106b9af3e6162eb4bdaf,1664447054000,abc.es,The firefighters of the Bejís fire have not ye...,"""The Central Independent Trade Union and Civil...",2
2,abc.es-9f18b7b37e3d1b287eafb09ace126d32629fa90a,1664448828000,abc.es,Leakage from Nord Stream is equivalent to the ...,""" The smoking pool of methane gas in the Balti...",1
3,abc.es-ad273960697997f76f5a4aaca52721d16cb2ab80,1664441927000,abc.es,"""Labour expels MP for saying UK Chancellor of ...","""The English MP Rupa Huq has been expelled as ...",1
4,abc.net.au-004a920f2038bba4473cfb627730f2fac36...,1656022155000,abc.net.au,"""'It's our history it's our everything': Farme...",Landowners says there was no consultation over...,0
...,...,...,...,...,...,...
51325,yahoo.com-fef60f9345dd2dbf9f120bf40d27e48edf72...,1654034016000,yahoo.com,Amber Heard's lawyers told jurors a verdict ag...,NaN,0
51326,yahoo.com-ff86305c30e24585534e1064dd52483aed71...,1653150583000,yahoo.com,A Maine town reportedly shut down after its on...,NaN,0
51327,yahoo.com-ffb99a82ff176686aa6cd70b5de3134418c0...,1652871195000,yahoo.com,"""High-profile candidates endorsed by Trump los...",NaN,0
51328,yahoo.com-ffcc76f8ebce23fee39d5df0965ec86924b6...,1653477680000,yahoo.com,China Protesters Demand Back Billions Invested...,NaN,0


In [ ]:
data=data.loc[data["confirmedLabel"]==2]

In [ ]:
keywords=pd.read_csv("/content/drive/MyDrive/Chain_News_Monitoring_Project/Pujit/keywords.csv")
keywords.fillna("",inplace=True)
keywords["Key word 1"]=keywords["Key word 1"].apply(lambda x: porter.stem(x))
keywords["Key word 2"]=keywords["Key word 2"].apply(lambda x: porter.stem(x))

In [ ]:
#findingt row no for words which need to occur together
idx=keywords.loc[keywords["Key word 1"]==""].index

In [ ]:
corpus_title = [preprocessing(title) for title in tqdm(list(data["title"]))]    
corpus_title 

In [ ]:
corpus = [preprocessing(title) for title in tqdm(list(data["content"]))]
corpus

In [ ]:
predictedlabel=[]
for i,l in tqdm(zip(corpus,corpus_title)):
  try:
    prediction=0
    for word in i:
       if word in list(keywords['Key word 1'][0:idx[0]]):
         prediction=2
         break
       elif word in list(keywords['Key word 1'][idx[0]+1:len(keywords)]):
          for word2 in i:
            if word2 in list(keywords['Key word 2'][idx[0]+1:len(keywords)]):
              prediction=2
              break
    predictedlabel.append(prediction)
  except:
    prediction=0
    for word in l:
       if word in list(keywords['Key word 1'][0:idx[0]]):
         prediction=2
         break
       elif word in list(keywords['Key word 1'][idx[0]+1:len(keywords)]):
          for word2 in l:
            if word2 in list(keywords['Key word 2'][idx[0]+1:len(keywords)]):
              prediction=2
              break
    predictedlabel.append(prediction)

732it [00:00, 1179.66it/s]


In [ ]:
data["predictedlabel"]=predictedlabel

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
confusion_matrix(data["confirmedLabel"],data["predictedlabel"])

array([[  0,   0],
       [423, 309]])